In [10]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from scipy import stats
import math
import statistics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize

%matplotlib inline

df_merged_venue = pd.read_csv('df_merged_venue.csv')

In [11]:
df_merged_venue = df_merged_venue.drop(columns=['Unnamed: 0','flat_type','month',
       'flat_model', 'planning_area','distance_to_nearest_primary_sch','distance_to_nearest_second_sch','distance_to_nearest_hawker'])

In [12]:
df_merged_venue['lease_commence_date'].unique()

array([1989, 2003, 2004, 1980, 1985, 1998, 2000, 1988, 1999, 1992, 1990,
       1993, 1984, 2013, 1975, 1995, 1997, 1994, 1986, 1978, 2005, 1974,
       1983, 1987, 2007, 2001, 1970, 1979, 1996, 1982, 2015, 1969, 2006,
       1981, 2010, 2002, 1968, 2016, 1966, 1991, 1976, 2009, 2012, 1971,
       1967, 1977, 2014, 1972, 1973, 2008, 2011, 2017, 2019, 2018])

In [18]:
df_merged_venue['street_name'].nunique()

1103

In [24]:
# df_train = pd.read_csv('cs5228-2020-semester-2-final-project/train.csv')
df_train[['latitude', 'longitude']].drop_duplicates().shape[0]

9138

In [27]:
df_train[['street_name','latitude', 'longitude']].drop_duplicates()

,street_name,latitude,longitude
0,pasir ris drive 4,1.369008,103.958697
1,punggol field,1.399007,103.906991
2,fernvale lane,1.388348,103.873815
3,clementi avenue 4,1.318493,103.766702
4,bukit batok street 11,1.348149,103.742658
...,...,...,...
421516,hougang avenue 1,1.354784,103.890318
421735,Sumang Walk,1.404949,103.896008
421818,Lengkong Tiga,1.324955,103.910175
422061,Marine Terrace,1.304372,103.914042


In [29]:
df_selected_lat_long = df_train.drop_duplicates(subset=['street_name'])
df_selected_lat_long = df_selected_lat_long[['street_name','latitude','longitude']]
scraped_venues = pd.read_csv('scraped_venues.csv')

In [113]:
list_old = list(scraped_venues['street_name'])
list_new = list(df_selected_lat_long['street_name'])
list_new = [x.lower() for x in list_new]
list_old = [x.lower() for x in list_old]
lat_old = list(scraped_venues['Latitude'])
lat_new = list(df_selected_lat_long['latitude'])
set_new = set(list_new)
list_new = list(set_new)
len(list_old), len(list_new)

(449, 553)

In [114]:
list_old = list(map(lambda st: str.replace(st, "ave ", "avenue "), list_old)) 
list_old = list(map(lambda st: str.replace(st, " st", " street"), list_old)) 
list_old = list(map(lambda st: str.replace(st, " rd", " road"), list_old)) 
list_old = list(map(lambda st: str.replace(st, " cres", " crescent"), list_old)) 
list_old = list(map(lambda st: str.replace(st, " streeth", " sth"), list_old)) 
list_old = list(map(lambda st: str.replace(st, " cl", " close"), list_old)) 

In [132]:
set_new = set(list_new)
set_old = set(list_old)
diff_list = list(set_new - set_old)
diff_list2 = list(set_old - set_new)
parsed_list = list(set_old - set(diff_list2))
len(parsed_list), len(diff_list)

(395, 158)

In [138]:
scraped_venues = pd.read_csv('scraped_venues.csv')
scraped_venues['street_name']= [x.lower() for x in scraped_venues['street_name']]
scraped_venues['street_name'] = scraped_venues['street_name'].str.replace("ave ", "avenue ", regex = True)
scraped_venues['street_name'] = scraped_venues['street_name'].str.replace(" st", " street", regex = True)
scraped_venues['street_name'] = scraped_venues['street_name'].str.replace(" rd", " road", regex = True)
scraped_venues['street_name'] = scraped_venues['street_name'].str.replace(" cres", " crescent", regex = True)
scraped_venues['street_name'] = scraped_venues['street_name'].str.replace(" streeth", " sth", regex = True)
scraped_venues['street_name'] = scraped_venues['street_name'].str.replace(" cl", " close", regex = True)

In [149]:
scraped_venues = scraped_venues.drop(columns=['Unnamed: 0'])

In [150]:
df_parsed = scraped_venues[scraped_venues['street_name'].isin(parsed_list)]

In [151]:
df_parsed

,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,ang mo kio avenue 10,1.367602,103.856844,23,18,23,5,17,8,2
1,ang mo kio avenue 4,1.382319,103.838706,10,5,13,6,11,5,0
2,ang mo kio avenue 5,1.376825,103.847556,12,13,18,4,10,1,1
3,ang mo kio avenue 1,1.364355,103.843642,15,12,9,4,17,6,0
4,ang mo kio avenue 3,1.369691,103.840906,19,12,11,7,15,6,1
...,...,...,...,...,...,...,...,...,...,...
443,elias road,1.373220,103.940290,4,11,7,6,10,3,0
444,holland close,1.307305,103.796033,68,38,60,4,54,25,1
445,tanglin halt road,1.300528,103.798360,35,24,26,9,23,11,0
447,wellington circle,1.452079,103.820354,29,14,33,6,24,10,0


In [152]:
df_to_parse = df_selected_lat_long[df_selected_lat_long['street_name'].isin(diff_list)]

In [155]:
df_to_parse.columns = ['street_name', 'Latitude','Longitude']
df_to_parse

,street_name,Latitude,Longitude
114,serangoon north avenue 3,1.371711,103.871234
124,simei road,1.344802,103.959862
126,tampines street 12,1.348070,103.943577
140,fernvale link,1.389799,103.880644
141,teban gardens road,1.321139,103.742788
...,...,...,...
64345,seng poh road,1.285297,103.833235
66377,french road,1.307362,103.860472
114508,tampines street 86,1.356709,103.931385
140702,jurong west central 3,1.340651,103.707506


In [ ]:
date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'SOVP55W3YHAD150G0TFX5LHBHBPNUCDW43FDVXDCLNDYAOHA'

names = df_to_parse['street_name']
latitudes = df_to_parse['Latitude']
longitudes = df_to_parse['Longitude']
radius = 500
limit = 100
venues_list = []

for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven.to_csv('new_vens.csv')

In [164]:
selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

In [166]:
selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven.to_csv('new_vens.csv')

In [167]:
selected_ven

,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,serangoon north avenue 3,1.371711,103.871234,21,17,20,3,14,5,0
1,simei road,1.344802,103.959862,8,7,7,10,6,2,0
2,tampines street 12,1.348070,103.943577,13,13,37,3,17,5,1
3,fernvale link,1.389799,103.880644,5,4,6,6,8,3,1
4,teban gardens road,1.321139,103.742788,6,8,10,7,6,3,0
...,...,...,...,...,...,...,...,...,...,...
113,montreal link,1.449943,103.826721,6,5,5,12,4,1,0
114,jurong east street 31,1.348038,103.729166,30,23,24,7,17,4,1
115,upper aljunied lane,1.333547,103.879066,17,13,13,5,13,4,0
116,sumang walk,1.406049,103.896180,4,2,11,0,3,0,0


In [172]:
df_to_parse = df_to_parse[118:]

In [173]:
date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'SOVP55W3YHAD150G0TFX5LHBHBPNUCDW43FDVXDCLNDYAOHA'

names = df_to_parse['street_name']
latitudes = df_to_parse['Latitude']
longitudes = df_to_parse['Longitude']
radius = 500
limit = 100

for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven.to_csv('new_vens.csv')

In [184]:
selected_ven.columns

Index(['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art',
       'Trending', 'Price_low', 'Price_high'],
      dtype='object')

In [176]:
df_all_parsed = selected_ven.append(df_parsed)
df_all_parsed

,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,serangoon north avenue 3,1.371711,103.871234,21,17,20,3,14,5,0
1,simei road,1.344802,103.959862,8,7,7,10,6,2,0
2,tampines street 12,1.348070,103.943577,13,13,37,3,17,5,1
3,fernvale link,1.389799,103.880644,5,4,6,6,8,3,1
4,teban gardens road,1.321139,103.742788,6,8,10,7,6,3,0
...,...,...,...,...,...,...,...,...,...,...
443,elias road,1.373220,103.940290,4,11,7,6,10,3,0
444,holland close,1.307305,103.796033,68,38,60,4,54,25,1
445,tanglin halt road,1.300528,103.798360,35,24,26,9,23,11,0
447,wellington circle,1.452079,103.820354,29,14,33,6,24,10,0


In [180]:
df_all_parsed.to_csv('df_venues_number.csv')

In [183]:
df_all_parsed[df_all_parsed['street_name']=='woodlands drive 16']

,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
368,woodlands drive 16,1.430358,103.799281,4,5,2,6,4,0,0
